# Installation

In [0]:
my_individual = 'NA20761' # HG00634

In [0]:
# 2-3 min.
%%capture
!apt install autoconf autogen
!git clone https://github.com/vcftools/vcftools.git
!cd vcftools && ./autogen.sh && ./configure && make && make install
!git clone https://github.com/samtools/htslib.git
!cd htslib && autoheader && autoconf && ./configure && make && make install
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20191028.zip -O plink.zip
!unzip plink.zip
!chmod +x plink
!git clone https://github.com/gonzalobenegas/CCB293_Health_Inference_Tutorial.git

In [0]:
# Extra installation steps not necessary today
#%%capture
#!git clone https://github.com/Ensembl/ensembl-vep
#%cd ensembl-vep
#!export PERL_MM_USE_DEFAULT=1 && cpan App::cpanminus
#!apt install mysql-server
#!apt install libmysqlclient-dev
#!cpanm DBI
#!cpanm Archive::Zip
#!cpanm DBD::mysql
#!cpanm Try::Tiny
# This can take up to an hour.
#!perl INSTALL.pl

In [0]:
# Download VCF for all 1000 Genomes individuals, chr16
# 2-3 min. (can take up to 10 min. occasionally)
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr16.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr16.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi

In [0]:
# load the libraries
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import colorsys
import seaborn as sns
import subprocess
from IPython.display import Image
import os
import seaborn as sns
sns.set(style="ticks")

# Exploring input data

In [0]:
# Individuals information
individuals = pd.read_csv('CCB293_Health_Inference_Tutorial/data/individuals.csv', '\t')
my_super_population = individuals.super_population[individuals.individual==my_individual].values[0]
my_population = individuals.population[individuals.individual==my_individual].values[0]
individuals

In [0]:
# Information about my individual
individuals[individuals.individual==my_individual]

In [0]:
# First lines of VCF file
!zcat ALL.chr16.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | head -n 300

In [0]:
# 10 minutes
# Filter only those variants with at least 1 copy in my individual
os.system('vcftools --gzvcf ALL.chr16.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz --indv {} --recode --out my_ind --non-ref-ac-any 1'.format(my_individual))

In [0]:
# First lines of my individual's VCF
# You can look at specific variant information in https://www.ncbi.nlm.nih.gov/snp/
!head -n 300 my_ind.recode.vcf

In [0]:
# Extract snp ids of my individual
!sed '/##/d' my_ind.recode.vcf | cut -f3 > my_ind.snps
my_ind_snps = pd.read_csv('my_ind.snps').values.ravel()

In [0]:
# Exercise: how many variants does my individual have on chromosome 16?
# How many variants are there in total in chromosome 16 among all individuals?

# Variant effect prediction

In [0]:
# Documentation about consequences:
# https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html
Image(filename='CCB293_Health_Inference_Tutorial/data/consequences.jpg', height=500)

In [0]:
# This can take several hours - it has already been precomputed
# VEP software documentation: https://uswest.ensembl.org/info/docs/tools/vep/script/vep_tutorial.html
#!./vep -i ALL.chr16.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz --cache --force_overwrite --sift b --canonical

In [0]:
# Loading precomputed results
!gunzip CCB293_Health_Inference_Tutorial/data/variant_effect_output.txt.gz

In [0]:
# Results file from VEP
# More information about specific variants: https://www.ncbi.nlm.nih.gov/snp/
!head -n 100 CCB293_Health_Inference_Tutorial/data/variant_effect_output.txt

In [0]:
# Load results into a Pandas DataFrame
variants_effect = pd.read_csv('CCB293_Health_Inference_Tutorial/data/variant_effect_output.txt', header=None, comment='#', delim_whitespace=True,
                   names=['Uploaded_variation',	'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'Extra'])
# Filter for variant effect predictions on the canonical transcript of each gene
variants_effect = variants_effect[variants_effect.Extra.str.contains('CANONICAL=YES')]
variants_effect

In [0]:
# Filtering for MC1R gene, a G protein-coupled receptor involved in pigmentation and associated with cancer
# https://en.wikipedia.org/wiki/Melanocortin_1_receptor
gene_variants_effect = variants_effect[variants_effect.Gene=='ENSG00000258839']
gene_variants_effect

In [0]:
# Histogram of MC1R variant consequences
sns.countplot(x='Consequence', data=gene_variants_effect)
plt.xticks(rotation=90);

In [0]:
# Filtering for MC1R variants in my individual
my_ind_gene_variants_effect = gene_variants_effect[gene_variants_effect.Uploaded_variation.isin(my_ind_snps)]
my_ind_gene_variants_effect

In [0]:
# Exercise: how many of these variants change the protein / amino acid sequence of MC1R? What are the protein changes?

In [0]:
# Exercise: pick one of your protein-altering variants and check whether your individual is homozygous or heterozygous for that variant.

In [0]:
# Histogram of MC1R variant consequences in my individual
sns.countplot(x='Consequence', data=my_ind_gene_variants_effect)
plt.xticks(rotation=90);

In [0]:
# Look for extra annotation in missense variants
my_ind_gene_variants_effect[my_ind_gene_variants_effect.Consequence=='missense_variant'].Extra.values

In [0]:
# Exercise: look at variants of my individual on HBA1, hemoglobin subunit alpha 1 (ENSG00000206172)

In [0]:
# Exercise: make a histogram of consequences (in canonical transcript) of type
# 'stop_gained', 'stop_lost', 'frameshift_variant', 'missense_variant', 'synonymous_variant', 'splice_acceptor_variant', 'splice_donor_variant', 'intron_variant'
# seen in all of chromosome 16 in the assigned individual

# Polygenic risk scores

In [0]:
# Downloading four polygenic risk scores for height, obtained from:
# https://github.com/msohail88/polygenic_selection/tree/master/polygenic_scores_pipeline
# The original GWAS summary statistics files used to develop the PRS are available at the following links:
# GIANT: https://portals.broadinstitute.org/collaboration/giant/index.php/GIANT_consortium_data_files#GIANT_Consortium_2012-2015_GWAS_Summary_Statistics
# UK Biobank (standing height = phenotype code 50): https://docs.google.com/spreadsheets/d/1b3oGI2lUt57BcuHttWaZotQcI0-mBRPyZihz87Ms_No/edit#gid=1209628142
%%capture
!wget https://www.dropbox.com/sh/qi80hwjusnrt0nz/AABJ-lpSUa018qWWp3NSEVnia?dl=1 -O tutorial_files.zip
!unzip tutorial_files.zip

In [0]:
# GWAS summary statistics from the GIANT study
# A1: effect allele
# A2: non-effect allele
# afa1: allele frequency of effect allele in GWAS population
# b: effect size (log-odds ratio per copy of effect allele)
# SE: standard error in effect size
# P: p-value (used for filtering)
# N: number of individuals in study
pd.read_csv('GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.0.01', delim_whitespace=True)

In [0]:
# GWAS summary statistics from the UK BioBank study
# tstat: t-statistic
pd.read_csv('50.assoc.tsv.processed.nodups.clumpedout.0.01', delim_whitespace=True)

In [0]:
# Exercise: what is the difference between the four scores? Do the number of SNPs in each score make sense?

In [0]:
# Exercise: describe the variant with the highest effect size in each score. Is it in a coding region? If so, in which gene?

In [0]:
# Scoring each individual
# ./plink --vcf <vcf_file> --score <GWAS summary statistics> <snp_id column> <effect allele column> <effect size column> <file_includes_header> <sum or average> --out <out_directory> 
# https://www.cog-genomics.org/plink/1.9/score
!./plink --vcf 1KG_snps_subsetted.vcf.gz --score GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.0.01 1 2 5 header sum --out giant_prs_0.01

In [0]:
!./plink --vcf 1KG_snps_subsetted.vcf.gz --score "GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.5E-8" 1 2 5 header sum --out "giant_prs_5E-8"

In [0]:
!./plink --vcf 1KG_snps_subsetted.vcf.gz --score "50.assoc.tsv.processed.nodups.clumpedout.0.01" 1 2 5 header sum --out "50_assoc_0.01"

In [0]:
 !./plink --vcf 1KG_snps_subsetted.vcf.gz --score "50.assoc.tsv.processed.nodups.clumpedout.5E-8" 1 2 5 header sum --out "50_assoc_5E-8"

In [0]:
# Output score for each individual
# PHENO = phenotype if available, -9 if not available
# CNT = total allele count
# CNT2 = number of effect alleles
# SCORESUM = polygenic score
pd.read_csv('giant_prs_0.01.profile', delim_whitespace=True)

In [0]:
# Loading results from the four scores into one DataFrame to plot results easily

def load_results(name):
    results = pd.read_csv(name+'.profile', delim_whitespace=True).merge(individuals, left_on='FID', right_on='individual')
    results['method'] = name
    # Standardizing the polygenic score
    results['standardized_score'] = (results.SCORESUM - results.SCORESUM.mean()) / results.SCORESUM.std()
    results['standardized_score_EUR'] = (results.SCORESUM - results.SCORESUM[results.super_population=='EUR'].mean()) / results.SCORESUM[results.super_population=='EUR'].std()
    results['standardized_score_my_super_population'] = (results.SCORESUM - results.SCORESUM[results.super_population==my_super_population].mean()) / results.SCORESUM[results.super_population==my_super_population].std()
    results['standardized_score_my_population'] = (results.SCORESUM - results.SCORESUM[results.population==my_population].mean()) / results.SCORESUM[results.population==my_population].std()
    return results

results = pd.concat([load_results(study) for study in ['giant_prs_0.01', 'giant_prs_5E-8', '50_assoc_0.01', '50_assoc_5E-8']])
# Defining population of my individual as 'My individual' to plot it separately
results.population[results.individual==my_individual] = 'My individual'
results.super_population[results.individual==my_individual] = 'My individual'    

In [0]:
# Plotting scores on the European populations
sns.boxplot(x="population", y="standardized_score_EUR",
            hue="method",
            data=results[results.super_population=='EUR']);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [0]:
# Plotting scores on my continental group
sns.boxplot(x="population", y="standardized_score_my_super_population",
            hue="method",
            data=results[results.super_population.isin([my_super_population, 'My individual'])]);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [0]:
# Plotting scores on my population
sns.boxplot(x="population", y="standardized_score_my_population",
            hue="method",
            data=results[results.population.isin([my_population, 'My individual'])]);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [0]:
# Comparing scores across continental groups
sns.boxplot(x="super_population", y="standardized_score",
            hue="method",
            data=results);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);